In [ ]:
import mbuild as mb
Poly = mb.lib.recipes.polymer

In [ ]:
# Quick example of the API and workflow
comp = mb.load('CC', smiles=True) # mBuild compound of the monomer unit
chain = Poly.Polymer()

chain.add_monomer(compound=comp,
                   bonding_indices=[2, -1],
                   separation=.15,
                   replace=True)

chain.add_end_groups(mb.load('C(=O)O',smiles=True), # Capping off this polymer with Carboxylic acid groups
                       bond_index=3,
                       separation=0.15)

chain.build(n=10, sequence='A')
chain.visualize().show()

In [ ]:
# Doing the same thing, but this time without adding an end group
# Now, the chain is capped with hydrogens (default behavior)

comp = mb.load('CC', smiles=True) # mBuild compound of the monomer unit
chain = Poly.Polymer()

chain.add_monomer(compound=comp,
                   bonding_indices=[2, -1],
                   separation=.15,
                   replace=True)

chain.build(n=10, sequence='A')
chain.visualize().show()

##  Here's an example with a more complicated monomer and a little more detail into what's going on  

SMILES strings for Poly-ether-ether-ketone (PEEK)  

One has para linkages, the other has meta  

Goal is to build up a polymer with alternating PARA-META monomers  

Also, just for fun, adding carboxylic acid end groups (ca)

In [ ]:
peek_para = mb.load("Oc1ccc(Oc2ccc(C(=O)c3ccccc3)cc2)cc1",smiles=True)
peek_meta = mb.load("Oc1cc(Oc2ccc(C(=O)c3ccccc3)cc2)ccc1", smiles=True)
ca = mb.load('C(=O)O', smiles=True)
peek_polymer = Poly.Polymer() # Create polymer instance

# For now, peek_polymer is an empty mBuild compound
print(peek_polymer)

# monomers and end_groups attributes: empty lists at the moment
print(peek_polymer.monomers)
print(peek_polymer.end_groups)

In [ ]:
# Use the add_monomer and add_end_group methods
# Pass in the compounds we want to use as monomers and end_groups
# The add_monomer function is called for each unique monomer type
# The add_end_groups funciton is called for the end group compound

peek_polymer.add_monomer(compound=peek_para,
                         bonding_indices = [22, 29],
                         separation = 0.1376,
                         replace=True
                        )

peek_polymer.add_monomer(compound=peek_meta,
                         bonding_indices = [22, 29],
                         separation = 0.1376,
                         replace=True
                        )

peek_polymer.add_end_groups(ca,
                            bond_index=3,
                            separation=0.13,
                            replace=True)

#At this point, peek_polymer is still an empty mBuild compound
#The monomers and end_groups attributes are no longer empty lists
print(peek_polymer)
print()
print(peek_polymer.monomers)
print()
print(peek_polymer.end_groups)

In [ ]:
# Now to actually make the polymer compound
# Essentially all of the currently exisitng Polymer() code was moved into a function called build
peek_polymer.build(n=3, sequence='AB')

# peek_polymer is no longer an empty compound
print(peek_polymer)

In [ ]:
peek_polymer.visualize().show()

## A look at what is actually happening with each of the class methods  

The `add_monomer` and `add_end_group` functions are handling the creation of ports.  

The key is in the `bond_indices` and `replace` parameters.
`bond_indices` points to the hydrogen atoms that are occupying the polymer bonding site and 
`replace` says to remove those atoms, and replace them with a port

When the port is created, it defaults to using the orientation that already existed between the hydrogen atom and the atom it was bonded to. 

In [ ]:
peek_para = mb.load("Oc1ccc(Oc2ccc(C(=O)c3ccccc3)cc2)cc1",smiles=True)
print('Before passing the compound into add_monomer()')
peek_para.visualize(show_ports=True).show()

peek_polymer = Poly.Polymer()

peek_polymer.add_monomer(compound=peek_para,
                         bonding_indices = [22, 29],
                         separation = 0.1376,
                         replace=True
                        )
print('After passing the compound into add_monomer()')
peek_polymer.monomers[0].visualize(show_ports=True).show()

In [ ]:
# Same thing with the end group
ca = mb.load('C(=O)O', smiles=True)
print('Before passing the compound into add_end_groups()')
ca.visualize(show_ports=True).show()

peek_polymer.add_end_groups(ca,
                            bond_index=3,
                            separation=0.13,
                            replace=True)

# ca[3] is the hydrogen bonded to the carbon atom

print('After passing the compound into add_end_groups()')
peek_polymer.end_groups[0].visualize(show_ports=True).show()

## Using replace=False instead
So far, all of the examples above used `replace=True` and the `bonding_indices` were the indices of hydrgogens that were being replaced by ports and removed to make room for the monomer-monomer bond.

I imagine this would be the most common work-flow for going straight from a SMILES string or compound file to a polymer, but it's possible use `replace=False`.  In this case, the atoms indicated in `bonding_indices` are the atoms forming the monomer-monomer bond.

Below is an example using the `ch2.pdb` file in the `moieties` directory.  In this case, we don't want to replace/remove any hydrogens, but add onto the carbon atom

# Reproducing the alkane chain
There currently exists a recipe that produces a simple alkane chain. Below I'll use the new `polymer.py` functionality to re-create the same alkane chain.

In [ ]:
ch2 = mb.load('lib/moieties/ch2.pdb')
chain = Poly.Polymer()
chain.add_monomer(ch2,
                  bonding_indices=[0, 0],
                  orientation=[[0, 1, 0], [0, -1, 0]],
                  separation=0.15,
                  replace=False)
chain.build(n=7, sequence='A')

chain.visualize()

In [ ]:
# It's super easy to throw different end groups on there

ch2 = mb.load('lib/moieties/ch2.pdb')
chain = Poly.Polymer()
chain.add_monomer(ch2,
                  bonding_indices=[0, 0],
                  orientation=[[0, 1, 0], [0, -1, 0]],
                  separation=0.15,
                  replace=False)

chain.add_end_groups(mb.load('c1ccccc1', smiles=True),
                    bond_index=-1,
                    separation=0.15,
                    replace=True)
chain.build(n=8, sequence='A')

chain.visualize().show()

## How we can use recipes

With this approach, to build up a polymer you would just need the following information:

1. A SMILES string (or a path to a compound file)
2. The indices of the hydrogen atoms to be removed
3. The bond length

So, having a library of this information to pull from could be a simple as a `.py` file with a dictionary, or a `.json` file


In [ ]:
monomer_dict = {
    
    'polyethylene': {'smiles': "CC",
               'bonding_indices': [2, -1],
               'bond_length': 0.1512
                    },
    
    'alkane': {'file': 'lib/moieties/ch2.pdb',
              'bonding_indices': [0, 0],
              'bond_length': 0.1512,
              'replace': False
              },
    
    'peek_para': {'smiles': "Oc1ccc(Oc2ccc(C(=O)c3ccccc3)cc2)cc1",
                   'bonding_indices': [22, 29],
                   'bond_length': 0.1376,
                  'description': "poly-ether-ether-ketone with a para configuration"                    
                 },
    
    'peek_meta': {'smiles': "Oc1cc(Oc2ccc(C(=O)c3ccccc3)cc2)ccc1",
                   'bonding_indices': [22, 29],
                   'bond_length': 0.1376,
                  'description': "poly-ether-ether-ketone with a meta configuration"
                 }
    
}

In [ ]:
# Maybe there is a from_recipe() function, for example:
polymer = Poly.Polymer.from_recipe(monomer='peek_para',
                                   n=10)